<a href="https://colab.research.google.com/github/hungviet0304/DS_Final_project/blob/master/DS_collect_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Crawling data**
 - Data source: https://www.imdb.com/
 - In 8 categories: action, comedy, Adventure,Animation, Crime, Romance, Documentary, Biography from 2010-10-10 to 2018-10-10
 - Total 80000 samples.
 - Total 900 samples to test
 - Details:
     + **Title**: name of the film.
     + **Runtime**: duration of films (**0** if unavailable).
     + **des**: plot of the movie.
     + **Date**: year (**0** if unavailable).
     + **Rating**: how well the film performed (**0** if unavailable).
     + **Genres**: genres of film.

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#%cd drive/My\ Drive/Colab\ Notebooks/DS_final
#!pwd

/content/drive/My Drive/Colab Notebooks/DS_final
/content/drive/My Drive/Colab Notebooks/DS_final


In [ ]:
import lxml
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import time

In [ ]:
def crawling(url_next_page, genre):

  position = 1
  url = url_next_page.format(genre, position).lower()
  print(url)
  
  page = get(url)
  s = BeautifulSoup(page.content,'lxml')

  #page title
  page_title = s.find("h1", class_ = "header").text.replace("\n","")
  body = s.find(id = 'main')
  content = body.find_all("div", class_ = "lister-item mode-advanced")
  total_results = 0

  df = pd.DataFrame(columns = ["title", "runtime","des","date","genre","cast","director","vote","gross","rating"])
  total_results = body.find("div",class_ = 'desc')
  total_results = total_results.find('span').text.replace(",","")
  total_results = int(re.findall(r'\d+',total_results)[-1])
  print("total results: {}".format(total_results))
  total_results = int(total_results)
  num_of_pages = total_results // 50 + 1 if total_results / 50 > total_results // 50 else total_results //50
  
  count = 0
  while True:

    for movie in content:
      
      tmp = movie.find("h3",class_ = "lister-item-header")
      movie_title=tmp.find("a").text

      try:
        movie_runtime = movie.find('span',class_ = 'runtime').text[:-4] # only number
      except:
        movie_runtime = 0
        # print("error at runtime: " + movie_title)

      try:
        movie_genre=movie.find('span',class_= 'genre').text.replace("\n","")
      except:
        movie_genre = ""
        # print("error at genre" + movie_title)
      
      

     # print(movie_title)
      if "\n" == (movie.find_all('p',class_ = "text-muted")[-1].next_element.next_element):
        try:
          movie_description = movie.find_all('p',class_ = "text-muted")[-1].text.replace("\n","")
          if (movie_description == "Add a Plot"):
            movie_description = ""
          
        except:
          # print("error at description: " + movie_title)
          movie_description = ""
      else:
        #print(movie.find_all('p',class_ = "text-muted")[-1].next_element.next_element.text)
        if (movie.find_all('p',class_ = "text-muted")[-1].next_element.next_element.text == "See full summary"):
        
          # access to https://www.imdb.com +"href" to get detail description
          href = movie.find_all('p',class_ = "text-muted")[-1].find_all('a')[-1].attrs["href"]
          tmp_link = "https://www.imdb.com" + href
          tmp_page = get(tmp_link)
          
          tmp_s = BeautifulSoup(tmp_page.content,'lxml')
          tmp_box_text = tmp_s.find("li",class_ = "ipl-zebra-list__item")    
          #print(tmp_box_text)
          #print("-------------")
          movie_description = tmp_box_text.find("p").text.replace("\n","") 
        else:
          movie_description = movie.find_all('p',class_ = "text-muted")[-1].text.replace("\n","")

      try:
        movie_date = tmp.find_all("span")[-1].text
        movie_date = int(re.findall(r'\d+',movie_date)[-1])
      except:
        movie_date = 0
        # print("error at date:" + movie_title)
      try:
        movie_rating = float(movie.find("strong").text)
      except:
        movie_rating = 0.0
        # print("error at rating: " + movie_title)


      #get from tail tab p
      tab_p = movie.find_all("p")


      if ("Vote" in tab_p[-1].text) or ("Gross" in tab_p[-1].text):
        vote_gross = tab_p[-1].text
        if ("Director" in tab_p[-2].text) or ("Star" in tab_p[-2].text):
          director_star = tab_p[-2].text
          try:
            vote_gross = vote_gross.replace("Votes:","")
            vote_gross = vote_gross.replace("Gross:","")
            vote_gross = vote_gross.split("|")
            
            if (len(vote_gross)==2):
              movie_vote = int(vote_gross[0].replace(",",""))
              movie_gross = vote_gross[1].strip()
            else:
              movie_vote = int(vote_gross[0].replace(",",""))
              movie_gross = ""
          except:
            movie_vote = 0
            movie_gross = ""


          director_star = director_star.replace("Director:","")
          director_star = director_star.replace("Directors:","")
          director_star = director_star.replace("Stars:","")
          director_star = director_star.replace("Star:","")
          director_star = director_star.split("|")
          try:

            if(len(director_star)==2):
              movie_director = director_star[0].strip().replace("\n","")
              movie_star = director_star[1].strip().replace("\n","")
              
            else:
              movie_star = director_star[0].strip().replace("\n","")
              movie_director = ""

          except:
            movie_director = ""
            movie_star = ""
        else:
          movie_director = ""
          movie_star = ""
          try:
            vote_gross = vote_gross.replace("Votes:","")
            vote_gross = vote_gross.replace("Gross:","")
            vote_gross = vote_gross.split("|")
            
            if (len(vote_gross)==2):
              movie_vote = int(vote_gross[0].replace(",",""))
              movie_gross = vote_gross[1].strip()
            else:
              movie_vote = int(vote_gross[0].replace(",",""))
              movie_gross = ""
          except:
            movie_vote = 0
            movie_gross = ""
        
      else:
        movie_gross = ""
        movie_vote = 0
        if ("Director" in tab_p[-1].text ) or ("Star" in tab_p[-1].text):
          director_star = tab_p[-1].text

          director_star = director_star.replace("Director:","")
          director_star = director_star.replace("Directors:","")
          director_star = director_star.replace("Star:","")
          director_star = director_star.replace("Stars:","")
          director_star = director_star.split("|")
          try:

            if(len(director_star)==2):
              movie_director = director_star[0].strip().replace("\n","")
              movie_star = director_star[1].strip().replace("\n","")
              
            else:
              movie_star = director_star[0].strip().replace("\n","")
              movie_director = ""

          except:
            movie_director = ""
            movie_star = ""
        else:
          movie_director = ""
          movie_star = ""

      
      

      df.loc[len(df)] = [movie_title, movie_runtime, movie_description,movie_date, movie_genre, movie_star, movie_director, movie_vote, movie_gross, movie_rating]
      if df.shape[0] >= 10000:
        # print()
        return df[:10000]
    position += 50
    url = url_next_page.format(genre,position).lower()
    #print(url)
    page = get(url)
    s = BeautifulSoup(page.content,'lxml')

    #page title
    page_title = s.find("h1", class_ = "header").text.replace("\n","")
    body = s.find(id = 'main')
    content = body.find_all("div", class_ = "lister-item mode-advanced")
    count += 1
    #print(count)
    # print(str(count) + " / " + str(num_of_pages))
    if count == num_of_pages:
      break

  return df

In [ ]:
def crawling_genre(url, genre):
  for g in genre:
    print("crawling " + g)
    df_tmp = crawling(url,g)
    print("crawed {}".format(df_tmp.shape[0]))
    df_tmp.to_csv("./data/" + g+".csv",index = False)


In [6]:
%%time
url = "https://www.imdb.com/search/title/?release_date=2010-10-10,2018-10-10&genres={}&languages=en&start={}&ref_=adv_nxt"
first_time = time.time()
crawling_genre(url,['Action','Comedy','Adventure','Animation','Crime','Romance','Documentary','Biography'])

second_time = time.time()
print("Total collecting time: ",second_time-first_time)

crawling Action
https://www.imdb.com/search/title/?release_date=2010-10-10,2018-10-10&genres=action&languages=en&start=1&ref_=adv_nxt
total results: 29927
crawed 10000
crawling Comedy
https://www.imdb.com/search/title/?release_date=2010-10-10,2018-10-10&genres=comedy&languages=en&start=1&ref_=adv_nxt
total results: 130086
crawed 10000
crawling Adventure
https://www.imdb.com/search/title/?release_date=2010-10-10,2018-10-10&genres=adventure&languages=en&start=1&ref_=adv_nxt
total results: 18986
crawed 10000
crawling Animation
https://www.imdb.com/search/title/?release_date=2010-10-10,2018-10-10&genres=animation&languages=en&start=1&ref_=adv_nxt
total results: 17853
crawed 10000
crawling Crime
https://www.imdb.com/search/title/?release_date=2010-10-10,2018-10-10&genres=crime&languages=en&start=1&ref_=adv_nxt
total results: 28385
crawed 10000
crawling Romance
https://www.imdb.com/search/title/?release_date=2010-10-10,2018-10-10&genres=romance&languages=en&start=1&ref_=adv_nxt
total results

- crawl test data
- Released between 2020-01-15 and 2022-01-15
- url example: https://www.imdb.com/search/title/?release_date=2020-01-10,&genres=action

In [ ]:
def crawling_genre_test(url, genre):
  for g in genre:
    print("crawling " + g)
    df_tmp = crawling(url,g)
    df_tmp = df_tmp.drop(columns="rating")
    print("crawed {}".format(df_tmp.shape[0]))
    df_tmp.to_csv("./test/" + g +".csv",index = False)


In [8]:
%%time
url = "https://www.imdb.com/search/title/?release_date=2020-01-15,2022-01-15&genres={}&languages=en&start={}&ref_=adv_nxt"
first_time = time.time()
crawling_genre_test(url,['Action','Comedy','Adventure','Animation','Crime','Romance','Documentary','Biography'])
second_time = time.time()
print("Total collecting time: ",second_time-first_time)

crawling Action
https://www.imdb.com/search/title/?release_date=2020-01-15,2022-01-15&genres=action&languages=en&start=1&ref_=adv_nxt
total results: 621
crawed 621
crawling Comedy
https://www.imdb.com/search/title/?release_date=2020-01-15,2022-01-15&genres=comedy&languages=en&start=1&ref_=adv_nxt
total results: 1202
crawed 1203
crawling Adventure
https://www.imdb.com/search/title/?release_date=2020-01-15,2022-01-15&genres=adventure&languages=en&start=1&ref_=adv_nxt
total results: 387
crawed 387
crawling Animation
https://www.imdb.com/search/title/?release_date=2020-01-15,2022-01-15&genres=animation&languages=en&start=1&ref_=adv_nxt
total results: 196
crawed 196
crawling Crime
https://www.imdb.com/search/title/?release_date=2020-01-15,2022-01-15&genres=crime&languages=en&start=1&ref_=adv_nxt
total results: 399
crawed 399
crawling Romance
https://www.imdb.com/search/title/?release_date=2020-01-15,2022-01-15&genres=romance&languages=en&start=1&ref_=adv_nxt
total results: 281
crawed 281
cr